#### Librerias

In [30]:
import pandas as pd
import librosa
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

#### Extraccion de Features

In [5]:
path_metadata = '../../fma_metadata'

raw_tracks = path_metadata + '/raw_tracks.csv'
tracks = path_metadata + '/tracks.csv'

In [6]:
df_tracks = pd.read_csv(raw_tracks)

In [7]:
batch_folder = 'batchs2'
csv_file = "mfcc_features.csv"

In [8]:
def load_batch(batch_filename):
    """
    Carga un archivo batch específico desde el directorio.

    Parámetros:
    - batch_filename: El nombre del archivo batch (.pkl)

    Devuelve:
    - Los datos almacenados en el archivo pickle (en este caso, una lista de vectores MFCC)
    """
    batch_path = os.path.join(batch_folder, batch_filename)
    print(batch_path)
    if not os.path.exists(batch_path):
        print(f"El archivo {batch_filename} no existe.")
        return None

    with open(batch_path, 'rb') as f:
        batch_data = pickle.load(f)

    return batch_data

In [53]:
batch_filename = 'mfcc_batch_0.pkl'  # Nombre del archivo del batch que quieres cargar
batch_data = load_batch(batch_filename)

batchs/mfcc_batch_0.pkl


In [58]:
def check_frame_sizes(batch_filename):
    batch_data = load_batch(batch_filename)

    if batch_data is None:
        return

    print(f"Total de canciones en el batch: {len(batch_data)}")

    # para las primeras 10 canciones
    for i in range(min(10, len(batch_data))):
        song_data = batch_data[i]
        print(f"Tamaño de los frames para la canción {i+1}: {song_data.shape}")

batch_filename = 'mfcc_batch_0.pkl'  # Cambia el nombre del archivo según sea necesario
check_frame_sizes(batch_filename)

batchs/mfcc_batch_0.pkl
Total de canciones en el batch: 1000


AttributeError: 'tuple' object has no attribute 'shape'

In [68]:
batch_data[0][1]

array([-1.22713936e+02,  1.17760078e+02, -4.23341751e+01,  3.80610008e+01,
       -2.30123215e+01,  2.39203796e+01, -1.19448633e+01,  1.36872883e+01,
       -7.29458094e+00,  4.04114103e+00, -4.85660362e+00, -1.66124272e+00,
       -6.30609512e+00, -2.19557142e+00,  1.73255253e+00, -4.33520377e-01,
        6.39691687e+00, -3.30403686e+00,  1.28376234e+00, -5.92810965e+00,
       -3.38252401e+00, -4.19097662e+00,  1.29010189e+00, -4.13347912e+00,
        6.23541977e-03, -3.76583219e+00, -7.17109084e-01,  3.40394266e-02,
       -8.24547350e-01,  3.55351877e+00,  1.48872995e+00,  3.06146622e+00,
        1.25891912e+00,  7.62736368e+00,  2.35857439e+00, -1.43467891e+00,
       -3.87765503e+00,  2.44992137e+00,  1.63306797e+00,  2.38041902e+00,
        3.25258327e+00,  6.39200151e-01, -7.23923683e-01, -5.35557747e-01,
       -1.84491944e+00, -1.21776676e+00,  2.26448464e+00, -1.31004977e+00,
       -2.16685629e+00, -3.17785192e+00,  1.32316804e+00, -5.86606145e-01,
       -5.93370795e-01, -

In [40]:
from scipy.signal import resample
import numpy as np

# Función para interpolar los vectores MFCC para que tengan el tamaño deseado
def interpolate_song(song_data, target_size=2582):
    current_size = song_data.shape[1]

    if current_size != target_size:
        new_song_data = np.zeros((song_data.shape[0], target_size))
        for i in range(song_data.shape[0]):  # Para cada coeficiente MFCC
            new_song_data[i] = resample(song_data[i], target_size)
        return new_song_data
    return song_data

# Función para procesar todo el batch con interpolación
def process_batch_for_analysis_with_interpolation(batch_filename, target_size=2582):
    batch_data = load_batch(batch_filename)

    if batch_data is None:
        return

    # Aplicar interpolación a cada canción en el batch
    processed_batch = []
    for song_data in batch_data:
        processed_song = interpolate_song(song_data, target_size)
        processed_batch.append(processed_song)

    return np.array(processed_batch)

def analyze_batch_with_interpolation(batch_filename):
    processed_batch = process_batch_for_analysis_with_interpolation(batch_filename)

    if processed_batch is None:
        return

    print(f"Tamaño del batch procesado: {processed_batch.shape}")

    # 3. Promediar sobre los frames (2582)
    batch_data_avg = np.mean(processed_batch, axis=2)
    print(f"Tamaño de los datos promediados: {batch_data_avg.shape}")

    # Estadísticas descriptivas
    mean_mfccs = np.mean(batch_data_avg, axis=0)
    std_mfccs = np.std(batch_data_avg, axis=0)

    print("Media de cada coeficiente MFCC:")
    print(mean_mfccs[:5])  # Muestra los primeros 5 coeficientes

    print("Desviación estándar de cada coeficiente MFCC:")
    print(std_mfccs[:5])  # Muestra los primeros 5 coeficientes

# Ejemplo de uso
batch_filename = 'mfcc_batch_0.pkl'  # Cambia el nombre del archivo según sea necesario
analyze_batch_with_interpolation(batch_filename)


batchs\mfcc_batch_0.pkl
Tamaño del batch procesado: (1000, 128, 2582)
Tamaño de los datos promediados: (1000, 128)
Media de cada coeficiente MFCC:
[-178.51670053  157.1441292   -29.06091298   34.02628143    0.61231745]
Desviación estándar de cada coeficiente MFCC:
[95.51986851 32.18492813 30.07797272 17.61256156 14.53879725]


In [48]:
from busquedas.KNN_HighD import *
from busquedas.KNN_RTree import *
from busquedas.KNN_Sequential import *

In [69]:
batch_folder = 'batchs/'

# Esta función cargará todos los archivos .pkl en la carpeta y devolverá una lista de tuplas (track_id, mfcc_vector)
def load_collection_from_pkl(batch_folder):
    collection = []
    for filename in os.listdir(batch_folder):
        if filename.endswith('.pkl'):
            file_path = os.path.join(batch_folder, filename)
            with open(file_path, 'rb') as f:
                batch_data = pickle.load(f)  # Cargar los datos del archivo .pkl
                for track_id, mfcc in batch_data:
                    collection.append((track_id, mfcc))
    return collection

# Cargar toda la colección desde los archivos .pkl
my_collection = load_collection_from_pkl(batch_folder)
print(f"Total de elementos en my_collection: {len(my_collection)}")

Total de elementos en my_collection: 24980


In [74]:
my_collection[10000]

('114072',
 array([-4.09685730e+02,  1.51876511e+02,  4.41475754e+01,  1.44969177e+01,
        -2.14168491e+01, -9.34521675e+00,  6.53687596e-01,  1.67928333e+01,
         1.64408169e+01,  1.51415110e+01,  1.49820125e+00, -4.74364901e+00,
        -6.58810043e+00, -3.68945026e+00,  6.91709742e-02,  3.73557448e+00,
         2.40533280e+00, -2.53010631e+00, -4.75862932e+00, -3.19729853e+00,
         1.67732871e+00,  3.89125562e+00,  4.02020979e+00, -1.82150871e-01,
        -4.03012228e+00, -4.59247112e+00,  1.32035315e+00,  6.39171696e+00,
         7.66038847e+00,  5.05359411e+00,  1.18540514e+00, -2.60636401e+00,
        -3.54237103e+00, -2.24659729e+00,  1.13600218e+00,  4.36398840e+00,
         3.27186465e+00, -9.46817458e-01, -5.54712296e+00, -7.37567949e+00,
        -3.36025214e+00,  2.54028463e+00,  6.15330935e+00,  6.63150263e+00,
         2.75476480e+00, -2.05115175e+00, -4.66844511e+00, -5.09661674e+00,
        -3.75743151e+00, -2.17037916e+00, -1.69065166e+00, -2.32793880e+00,
 

In [75]:
# Suponiendo que tienes un vector query_mfcc para buscar
query_mfcc = np.random.rand(128, 2582)  # Ejemplo, reemplazar con un MFCC real
query_mfcc = np.mean(query_mfcc, axis=1)  # Promedio de los coeficientes MFCC para reducir dimensionalidad

# Crear una instancia de la clase KNN
knn = KNN_HighD(num_bits=64, collection=my_collection)  # Usar la colección procesada con interpolación

# Realizar la consulta
resultados = knn.knn_query(query_mfcc, k=5)  # Consultar los 5 más similares
print(resultados)


[('087638', 110.88671), ('116825', 129.2381), ('102133', 140.73315), ('009505', 154.19804), ('129701', 237.41994)]
